In [1]:
using TensorOperations, TensorKit
import TensorKit.×
using FLoops
include("./iPEPS_Fermionic/iPEPS.jl")
include("./CTMRG_Fermionic/CTMRG.jl")
include("./Cal_Obs_Fermionic/Cal_Obs.jl")
include("./models/Hubbard_Z2SU2.jl")
include("./models/tJ_Z2.jl")
@show pkgversion(TensorKit)
using JLD2, HDF5
using Strided
Strided.enable_threads()

pkgversion(TensorKit) = v"0.12.3"


4

In [ ]:
# pspace = GradedSpace{fSU₂}((-1, 0) => 1, (0, 1 // 2) => 1, (1, 0) => 1)
pspace = Rep[U₁×SU₂]((-1, 0) => 1, (0, 1 // 2) => 1, (1, 0) => 1)
aspace = Rep[U₁×SU₂]((2, 0) => 2)

# aspacel = aspacer = aspacet = aspaceb = pspace
@show pspace
nothing

In [ ]:
pspace = Rep[ℤ₂×SU₂]((0, 0) => 2, (1, 1 // 2) => 1)
# pspace = Rep[ℤ₂]((1 => 1), (0 => 2))
aspacel = Rep[ℤ₂×SU₂]((0, 0) => 2, (1, 1 // 2) => 1)
# aspacer = Rep[ℤ₂×SU₂]((0, 0) => 2, (0, 1) => 2)
aspacer = aspacel
aspacet = Rep[ℤ₂×SU₂]((0, 0) => 2, (1, 1 // 2) => 1)
# aspaceb = Rep[ℤ₂×SU₂]((0, 0) => 2, (1, 3 // 2) => 2)
aspaceb = aspacet

# A = TensorMap(randn, ComplexF64, aspacel ⊗ aspacet ⊗ pspace, aspacer ⊗ aspaceb)
# Ms = [A, A, A, A]
# ipeps = iPEPS(Ms, 2, 2)
# Abar = bar(A)
# @show space(Abar)
# gate1 = swap_gate(space(Abar)[4], space(Abar)[5])
# @show space(gate1)
# gate2 = swap_gate(space(Abar)[1], space(Abar)[2])
# @show space(gate2)
# gate3 = swap_gate(space(A)[4], space(A)[5])
# @show space(gate3)
# gate4 = swap_gate(space(A)[1], space(A)[2])
# @show space(gate4)
# gate5 = swap_gate(space(A)[1], space(Abar)[2])
# @show space(gate5)
@assert pspace isa ElementarySpace
@show fieldnames(GradedSpace)

In [ ]:
include("./iPEPS_Fermionic/iPEPS.jl")
include("./CTMRG_Fermionic/CTMRG.jl")
include("./models/tJ_Z2.jl")
include("./simple_update_Fermionic/simple_update.jl")
include("./fast_full_update_Fermionic/fast_full_update.jl")
include("./Cal_Obs_Fermionic/Cal_Obs.jl")

ipeps, envs, para = load("/home/tcmp2/JuliaProjects/tJZ2_Lx2Ly2_SU_t3.0t'0.51J1.0J'0.0289h0.6mu5.2_ipepsEnv_D6chi140.jld2", "ipeps", "envs", "para")
Lx = ipeps.Lx
Ly = ipeps.Ly
@show para[:χ]
ipepsbar = bar(ipeps)

Obs2si = Cal_Obs_2site(ipeps, ipepsbar, envs, ["Δₛ", "Δₛdag"], para; site1=[1, 1], site2=[0, 2], get_op=get_op_tJ)

nothing

In [ ]:
pspace = Rep[ℤ₂](0 => 1, 1 => 2)
include("./models/tJ_Z2.jl")
OpZ = Z(pspace)

Fdag₊, F₊ = FdagF₊(pspace)
@tensor fdagf₊[p1, p3; p2, p4] := OpZ[p1, p1in] * Fdag₊[p1in, p2, a] * F₊[a, p3, p4]
@show Fdag₊
Fdag₋, F₋ = FdagF₋(pspace)
@show Fdag₋
@tensor fdagf₋[p1, p3; p2, p4] := Fdag₋[p1, p2, a] * F₋[a, p3in, p4] * OpZ[p3, p3in]

F₊, Fdag₊ = FFdag₊(pspace)
@tensor ffdag₊[p1, p3; p2, p4] := OpZ[p1, p1in] * F₊[p1in, p2, a] * Fdag₊[a, p3, p4]
F₋, Fdag₋ = FFdag₋(pspace)
@tensor ffdag₋[p1, p3; p2, p4] := F₋[p1, p2, a] * Fdag₋[a, p3in, p4] * OpZ[p3, p3in]

@show ishermitian(fdagf₊ - ffdag₊ + fdagf₋ - ffdag₋)

In [ ]:
include("./models/Hubbard_Z2SU2.jl")
pspace = Rep[ℤ₂×SU₂]((0, 0) => 2, (1, 1 // 2) => 1)

OpZ = Z(pspace)
Opnd = nd(pspace)
Opn = n(pspace)
OpI = id(pspace)
Fdag, F = FdagF(pspace)
@tensor fdagf[p1, p3; p2, p4] := Fdag[p1in, p2, a] * F[a, p3, p4]
# @tensor fdagf[p1, p3; p2, p4] := Fdag[p1, p2, a] * F[a, p3, p4]

F, Fdag = FFdag(pspace)
@tensor ffdag[p1, p3; p2, p4] := F[p1in, p2, a] * Fdag[a, p3, p4]

@show ishermitian(fdagf - ffdag)

In [2]:
ipeps, envs, para = load("/home/tcmp2/JuliaProjects/tJZ2_Lx2Ly2_SU_t3.0t'0.51J1.0J'0.0289h0.6mu5.2_ipepsEnv_D6chi140.jld2", "ipeps", "envs", "para")
ipepsbar = bar(ipeps)
Lx = 2
Ly = 2
# 计算观测量
println("============== Calculating Obs ====================")
site1Obs = ["N", "Sz"]             # 计算这些单点观测量
site2Obs = ["hijNN", "SpSm", "SzSz", "NN", "Δₛ", "Δₛdag"]   # 计算这些两点观测量
site2Obsdiag = ["hijNNN", "SpSm", "SzSz", "NN"]

rslt1s = Vector{Dict}(undef, Lx*Ly)
rslt2s_h = Vector{Dict}(undef, Lx*Ly)
rslt2s_v = Vector{Dict}(undef, Lx*Ly)
rslt2s_lu2rd = Vector{Dict}(undef, Lx*Ly)
rslt2s_ru2ld = Vector{Dict}(undef, Lx*Ly)
# rslt = Dict(Gates[ind] => (vals[ind] / nrm) for ind in 1:length(vals))
@floop for (ind, val) in enumerate(CartesianIndices((Lx, Ly)))
    (xx, yy) = Tuple(val)
    Obs1si = Cal_Obs_1site(ipeps, ipepsbar, envs, site1Obs, para; site=[xx, yy], get_op=get_op_tJ)
    @show (xx, yy, Obs1si)
    @reduce filling += get(Obs1si, "N", NaN)
    @reduce magnetization += abs(get(Obs1si, "Sz", NaN))
    rslt1s[ind] = Obs1si

    Obs2si_h = Cal_Obs_2site(ipeps, ipepsbar, envs, site2Obs, para; site1=[xx, yy], site2=[xx + 1, yy], get_op=get_op_tJ)
    @show (xx, yy, Obs2si_h)
    rslt2s_h[ind] = Obs2si_h
    Obs2si_v = Cal_Obs_2site(ipeps, ipepsbar, envs, site2Obs, para; site1=[xx, yy], site2=[xx, yy + 1], get_op=get_op_tJ)
    @show (xx, yy, Obs2si_v)
    rslt2s_v[ind] = Obs2si_v
    @reduce Eg += (get(Obs2si_h, "hijNN", NaN) + get(Obs2si_v, "hijNN", NaN))

    Obs2sidiag_lurd = Cal_Obs_2site(ipeps, ipepsbar, envs, site2Obsdiag, para; site1=[xx, yy], site2=[xx + 1, yy + 1], get_op=get_op_tJ)
    rslt2s_lu2rd[ind] = Obs2sidiag_lurd
    Obs2sidiag_ruld = Cal_Obs_2site(ipeps, ipepsbar, envs, site2Obsdiag, para; site1=[xx, yy], site2=[xx - 1, yy + 1], get_op=get_op_tJ)
    rslt2s_ru2ld[ind] = Obs2sidiag_ruld
    @reduce Eg += (get(Obs2sidiag_lurd, "hijNNN", NaN) + get(Obs2sidiag_ruld, "hijNNN", NaN))
    GC.gc()
end
filling = filling / (Lx * Ly)
@show filling
magnetization = magnetization / (Lx * Ly)
@show magnetization
Eg = Eg / (Lx * Ly) + filling * para[:μ]
@show Eg

============== Calculating Obs ====================
Fermionic! Calculating ["N", "Sz"] at site [2, 1]
Fermionic! Calculating ["N", "Sz"] at site [2, 2]
Fermionic! Calculating ["N", "Sz"] at site [1, 2]
Fermionic! Calculating ["N", "Sz"] at site [1, 1]
(xx, yy, Obs1si) = (1, 1, Dict("N" => 0.9102613967538491, "Sz" => -0.2273770224773779))
(xx, yy, Obs1si) = (1, 2, Dict("N" => 0.9130966246164545, "Sz" => 0.3110148664730846))
(xx, yy, Obs1si) = (2, 2, Dict("N" => 0.9002661026595117, "Sz" => -0.1702414731408852))
(xx, yy, Obs1si) = (2, 1, Dict("N" => 0.9132360893926827, "Sz" => 0.3109635828057502))
Fermionic! Calculating ["hijNN", "SpSm", "SzSz", "NN", "Δₛ", "Δₛdag"] at site [2, 2] and site [3, 2]
Fermionic! Calculating ["hijNN", "SpSm", "SzSz", "NN", "Δₛ", "Δₛdag"] at site [1, 2] and site [2, 2]
Fermionic! Calculating ["hijNN", "SpSm", "SzSz", "NN", "Δₛ", "Δₛdag"] at site [2, 1] and site [3, 1]
Fermionic! Calculating ["hijNN", "SpSm", "SzSz", "NN", "Δₛ", "Δₛdag"] at site [1, 1] and site [

4.683668644630509

In [24]:
# =================== save Obs to file ================================
Obsname = joinpath("/home/tcmp2/JuliaProjects/", "Obs.h5")
f = h5open(Obsname, "w")
T = eltype(ipeps[1, 1])
try 
    for obs in site1Obs  # eg: obs == "N"
        tmp = Matrix{T}(undef, Lx*Ly, 3)  # 注意！！这里必须是默认数据类型才能存进去. 如涉及虚数观测量, 坐标会存虚数.
        for (ind, val) in enumerate(CartesianIndices((Lx, Ly)))
            (xx, yy) = Tuple(val)
            for Obs1si in rslt1s  # Obs1si is a Dict. 记录在某个site的所有单点观测量值
                tmp[ind, 1], tmp[ind, 2] = xx, yy
                tmp[ind, 3] = get(Obs1si, obs, NaN)
            end
        end
        write(f, obs, sortslices(tmp, dims=1))
    end
    for obs in site2Obs  # eg: obs == "NN"
        tmp_h = Matrix{T}(undef, Lx*Ly, 5)
        tmp_v = Matrix{T}(undef, Lx*Ly, 5)
        for (ind, val) in enumerate(CartesianIndices((Lx, Ly)))
            (xx, yy) = Tuple(val)
            for Obs2si_h in rslt2s_h  # Obs2si is a Dict. 记录在某个site的所有两点观测量值
                tmp_h[ind, 1], tmp_h[ind, 2], tmp_h[ind, 3], tmp_h[ind, 4] = xx, yy, xx+1-Int(ceil((xx+1)/Lx)-1)*Lx, yy-Int(ceil(yy/Ly)-1)*Ly
                tmp_h[ind, 5] = get(Obs2si_h, obs, NaN)
            end
            for Obs2si_v in rslt2s_v  # Obs2si is a Dict. 记录在某个site的所有两点观测量值
                tmp_v[ind, 1], tmp_v[ind, 2], tmp_v[ind, 3], tmp_v[ind, 4] = xx, yy, xx-Int(ceil(xx/Lx)-1)*Lx, yy+1-Int(ceil((yy+1)/Ly)-1)*Ly
                tmp_v[ind, 5] = get(Obs2si_v, obs, NaN)
            end
        end
        write(f, obs, sortslices(vcat(tmp_h, tmp_v), dims=1))
    end
    for obs in site2Obsdiag  # eg: obs == "NN"
        tmp_lurd = Matrix{T}(undef, Lx*Ly, 5)
        tmp_ruld = Matrix{T}(undef, Lx*Ly, 5)
        for (ind, val) in enumerate(CartesianIndices((Lx, Ly)))
            (xx, yy) = Tuple(val)
            for Obs2si_lurd in rslt2s_lu2rd  # Obs2si is a Dict. 记录在某个site的所有两点观测量值
                tmp_lurd[ind, 1], tmp_lurd[ind, 2], tmp_lurd[ind, 3], tmp_lurd[ind, 4] = xx, yy, xx+1-Int(ceil((xx+1)/Lx)-1)*Lx, yy+1-Int(ceil((yy+1)/Ly)-1)*Ly
                tmp_lurd[ind, 5] = get(Obs2si_lurd, obs, NaN)
            end
            for Obs2si_ruld in rslt2s_ru2ld  # Obs2si is a Dict. 记录在某个site的所有两点观测量值
                tmp_ruld[ind, 1], tmp_ruld[ind, 2], tmp_ruld[ind, 3], tmp_ruld[ind, 4] = xx, yy, xx-1-Int(ceil((xx-1)/Lx)-1)*Lx, yy+1-Int(ceil((yy+1)/Ly)-1)*Ly
                tmp_ruld[ind, 5] = get(Obs2si_ruld, obs, NaN)
            end
        end
        write(f, string(obs, "diag"), sortslices(vcat(tmp_lurd, tmp_ruld), dims=1))
    end
finally
    close(f)
end

tmp_h = [1.0 1.0 2.0 1.0 -3.159497101565441; 2.0 1.0 1.0 1.0 -3.159497101565441; 1.0 2.0 2.0 2.0 -3.159497101565441; 2.0 2.0 1.0 2.0 -3.159497101565441]
tmp_v = [1.0 1.0 1.0 2.0 -3.203850755090258; 2.0 1.0 2.0 2.0 -3.203850755090258; 1.0 2.0 1.0 1.0 -3.203850755090258; 2.0 2.0 2.0 1.0 -3.203850755090258]
tmp_h = [1.0 1.0 2.0 1.0 -0.10966046026832667; 2.0 1.0 1.0 1.0 -0.10966046026832667; 1.0 2.0 2.0 2.0 -0.10966046026832667; 2.0 2.0 1.0 2.0 -0.10966046026832667]
tmp_v = [1.0 1.0 1.0 2.0 -0.12477753228642627; 2.0 1.0 2.0 2.0 -0.12477753228642627; 1.0 2.0 1.0 1.0 -0.12477753228642627; 2.0 2.0 2.0 1.0 -0.12477753228642627]
tmp_h = [1.0 1.0 2.0 1.0 -0.08138132802811247; 2.0 1.0 1.0 1.0 -0.08138132802811247; 1.0 2.0 2.0 2.0 -0.08138132802811247; 2.0 2.0 1.0 2.0 -0.08138132802811247]
tmp_v = [1.0 1.0 1.0 2.0 -0.0915327969576596; 2.0 1.0 2.0 2.0 -0.0915327969576596; 1.0 2.0 1.0 1.0 -0.0915327969576596; 2.0 2.0 2.0 1.0 -0.0915327969576596]
tmp_h = [1.0 1.0 2.0 1.0 0.8191381506193681; 2.0 1.0 1